In [1]:
import pandas as pd
import numpy as np
import bluebelt as blue
import datetime

In [2]:
nrows = 100
ncols = 4

np.random.seed(27)
series = pd.Series(np.random.weibull(7,nrows) * 100, index=pd.date_range(start='2019-12-06', periods=nrows), name='process A')
frame = pd.DataFrame(np.random.normal(loc=100, scale=20, size=ncols*nrows).reshape(nrows, ncols), index=pd.date_range(start='1-1-2020', periods=nrows), columns=[f'process {col}' for col in range(1, ncols+1)])

series[series.sample(5).index] = np.nan # break the series
for col in frame.columns:
    frame.at[frame.sample(5).index, col] = np.nan # break the frame

In [20]:
x = series.blue.index.groupby(level='day')
x.sum()

day
1    1283.859601
2    1249.784104
3    1422.188574
4    1172.497273
5    1401.281635
6    1241.372457
7    1378.581886
Name: process A, dtype: float64

In [25]:
x.grouped.obj.index.names[-1] == 'day'[-1]

False

In [32]:
xxx = ['week', 'day', 't']
xxx[-1] if isinstance(xxx, list) else xxx

't'

In [4]:
series.dropna().index.blue.alt

array([ 0.,  1.,  2.,  3.,  5.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
       55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67.,
       68., 69., 70., 71., 72., 73., 74., 75., 76., 77., 79., 81., 82.,
       83., 84., 85., 86., 87., 88., 89., 90., 91., 92., 93., 94., 95.,
       96., 97., 98., 99.])

In [3]:
s = series.blue.index.groupby(level=['year', 'week', 'day']).sum()
blue.core.index.IndexToolkit(s.index).alt

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 95., 96., 97., 98., 99.])

In [5]:
series.index.blue.iso()

MultiIndex([(2019, 49, 5),
            (2019, 49, 6),
            (2019, 49, 7),
            (2019, 50, 1),
            (2019, 50, 2),
            (2019, 50, 3),
            (2019, 50, 4),
            (2019, 50, 5),
            (2019, 50, 6),
            (2019, 50, 7),
            (2019, 51, 1),
            (2019, 51, 2),
            (2019, 51, 3),
            (2019, 51, 4),
            (2019, 51, 5),
            (2019, 51, 6),
            (2019, 51, 7),
            (2019, 52, 1),
            (2019, 52, 2),
            (2019, 52, 3),
            (2019, 52, 4),
            (2019, 52, 5),
            (2019, 52, 6),
            (2019, 52, 7),
            (2020,  1, 1),
            (2020,  1, 2),
            (2020,  1, 3),
            (2020,  1, 4),
            (2020,  1, 5),
            (2020,  1, 6),
            (2020,  1, 7),
            (2020,  2, 1),
            (2020,  2, 2),
            (2020,  2, 3),
            (2020,  2, 4),
            (2020,  2, 5),
            (2020,  2, 6),
 

In [30]:
def _get_alt_index(index):
    
    if not isinstance(index, pd.MultiIndex):
        raise ValueError("index is not a Pandas MultiIndex")
        
    index = _complete(index)
    
    if 'week' in index.names:
        _index = np.array([datetime.datetime.fromisocalendar(*row).timestamp() for row in index], dtype='int')
    
    elif 'month' in index.names: 
        _index = np.array([datetime.datetime(*row).timestamp() for row in index], dtype='int')
    
    # clean _index
    _index = _index - _index.min()
    _index = _index / np.gcd.reduce(_index)
    
    return _index

def _complete(index):
    '''
    add missing levels to the index
    '''
    
    if not isinstance(index, pd.MultiIndex):
        raise ValueError("index is not a Pandas MultiIndex")
        
    if 'week' in index.names:
        # add missing levels to the index
        _dict = index.to_frame(index=False).to_dict()

        _default_dict ={
            'year': {key: datetime.datetime.now().year for key in range(index.shape[0])},
            'week': {key: 1 for key in range(index.shape[0])},
            'day': {key: 1 for key in range(index.shape[0])},
        }

        for level, data in _default_dict.items():
            _dict[level] = _dict.get(level, _default_dict[level])
            
        return pd.MultiIndex.from_frame(pd.DataFrame.from_dict(_dict)[_default_dict.keys()])
    
    elif 'month' in index.names:
        _dict = index.to_frame(index=False).to_dict()

        _default_dict = {
            'year': {key: datetime.datetime.now().year for key in range(index.shape[0])},
            'month': {key: 1 for key in range(index.shape[0])},
            'day': {key: 1 for key in range(index.shape[0])},
        }

        for level, data in _default_dict.items():
            _dict[level] = _dict.get(level, _default_dict[level])
        return pd.MultiIndex.from_frame(pd.DataFrame.from_dict(_dict)[_default_dict.keys()])
    
    else:
        return index

In [31]:
_get_alt_index(s.dropna().index)

array([ 0.,  1.,  2.,  3.,  5.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
       15., 16., 17., 18., 19., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
       55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67.,
       68., 69., 70., 71., 72., 73., 74., 75., 76., 77., 79., 81., 82.,
       83., 84., 85., 86., 87., 88., 89., 90., 91., 92., 93., 94., 95.,
       96., 97., 98., 99.])

In [17]:
_levels = {
        'year': 0,
        'month': 1,
        'week': 2,
        'day': 3,
        'hour': 4,
        'minute': 5,
        'second': 6,
    }

keys = ['minute', 'year', 'week']
_levels = {key: _levels[key] for key in keys }
max_key = max(_levels, key=_levels.get)
max_key

'minute'

In [24]:
level = 'week'

In [25]:
_levels = ['year', 'week', 'day']

level = _levels[:_levels.index(level)+1]

In [26]:
level

['year', 'week']

In [21]:
_levels.index('week')

1